## Tensorflow RNN text generator network (baseline)
Based on the following text-generation tutorial from Tensorflow (https://www.tensorflow.org/tutorials/text/text_generation) 

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time
import pandas as pd

### Prepare dataset

In [ ]:
import pandas as pd
legitimate_tweets_df = pd.read_csv("/content/drive/Shareddrives/CSCI 5523 Group Project/Data/legitimate_users_tweets_filtered.csv",header=0)
tweets = legitimate_tweets_df['tweet']
tweets = tweets[tweets.notna()]
#tweets.to_string("/content/drive/Shareddrives/CSCI 5523 Group Project/Data/GAN data/legitimate_users_tweets_only.txt", index = False)

In [ ]:
legitimate_tweets_df

,user_id,tweet_id,tweet,created_at,label
0,614,5873834688,I wish I had more free time. I'd LOVE to see you!,2009-11-19 18:16:40,0
1,614,5873809295,"Tonight, tomorrow. On the plane at 5 pm.",2009-11-19 18:15:42,0
2,614,5291252160,"I'm at Carlucci's in Salt Lake City, UT http:/...",2009-10-30 11:24:52,0
3,614,5205651441,@spam @JannetteDavid,2009-10-27 12:17:35,0
4,1038,5762418891,@dialupkid Mijn vriendin en ik hebben een geza...,2009-11-16 05:08:29,0
...,...,...,...,...,...
2410894,93390990,6169834217,Earned the Tweeter Achievement! #epicpetwars h...,2009-11-29 07:39:54,0
2410895,93390990,6168916131,meet me on ELIMINATE pro!!,2009-11-29 06:44:03,0
2410896,93402679,6170059145,my twitter,2009-11-29 07:52:22,0
2410897,93419256,6171947104,exploring this thing...,2009-11-29 09:25:53,0


In [ ]:
# sample 10% of tweets, remove NA entries
legitimate_tweets_df_sample = legitimate_tweets_df.sample(legitimate_tweets_df.shape[0]//10)
tweets_sample = legitimate_tweets_df_sample['tweet']
tweets_sample = tweets_sample[tweets_sample.notna()]

# save legitimate user tweets as text file
path = "/content/legitimate_users_tweets_only_sample.txt"
tweets_sample.to_string(path, index=False)

In [ ]:
# load text
#text = open(path, 'rb').read().decode(encoding = 'utf-8')
text = open("/content/drive/Shareddrives/CSCI 5523 Group Project/Data/GAN data/legitimate_users_tweets_only.txt",'rb').read().decode(encoding = 'utf-8')

In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

 I wish I had more free time. I'd LOVE to see you!
          Tonight, tomorrow. On the plane at 5 pm.
 I'm at Carlucci's in Salt Lake City, UT http:/...
                              @spam @JannetteDavid
 @dialupkid Mijn vriendin en ik hebben een gez


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

97 unique characters


## Text Vectorization

Convert strings to a numerical representation.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

ids = ids_from_chars(chars)
ids

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

chars = chars_from_ids(ids)
chars

Use `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(122955848,), dtype=int64, numpy=array([ 3, 44,  3, ..., 17, 17, 17])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

 
I
 
w
i
s
h
 
I
 


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b' ' b'I' b' ' b'w' b'i' b's' b'h' b' ' b'I' b' ' b'h' b'a' b'd' b' '
 b'm' b'o' b'r' b'e' b' ' b'f' b'r' b'e' b'e' b' ' b't' b'i' b'm' b'e'
 b'.' b' ' b'I' b"'" b'd' b' ' b'L' b'O' b'V' b'E' b' ' b't' b'o' b' '
 b's' b'e' b'e' b' ' b'y' b'o' b'u' b'!' b'\n' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b'T' b'o' b'n' b'i' b'g' b'h' b't' b',' b' '
 b't' b'o' b'm' b'o' b'r' b'r' b'o' b'w' b'.' b' ' b'O' b'n' b' ' b't'
 b'h' b'e' b' ' b'p' b'l' b'a' b'n' b'e' b' ' b'a' b't' b' ' b'5' b' '
 b'p' b'm' b'.'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b" I wish I had more free time. I'd LOVE to see you!\n          Tonight, tomorrow. On the plane at 5 pm."
b"\n I'm at Carlucci's in Salt Lake City, UT http:/...\n                              @spam @JannetteDavi"
b'd\n @dialupkid Mijn vriendin en ik hebben een geza...\n Weet iemand wie de Amsterdamse screening van 1.'
b'..\n Gat in de markt: slow flying. RT @MarcDerby Wi...\n @erwblo Het woord gratis doet hierbij wonderen'
b'...\n         Christmas Station http://flic.kr/p/7g5wYq\n Vier vlaggen halfstok bij de politie op de Br'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b" I wish I had more free time. I'd LOVE to see you!\n          Tonight, tomorrow. On the plane at 5 pm"
Target: b"I wish I had more free time. I'd LOVE to see you!\n          Tonight, tomorrow. On the plane at 5 pm."


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 256

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 99) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  25344     
_________________________________________________________________
gru_2 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_2 (Dense)              multiple                  101475    
Total params: 4,065,123
Trainable params: 4,065,123
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([73, 58, 21, 94, 19, 52, 58, 98, 79, 56, 28,  2, 81, 57, 89,  1,  9,
       71, 83, 37, 77, 63, 18,  8, 25, 63, 13,  1, 33, 65, 14, 66, 67,  4,
       18, 92, 11, 73, 11, 97, 81, 51, 53, 88,  9, 71, 69, 55,  1, 28, 51,
       79, 28, 44, 10, 49, 30, 27, 65, 39, 65, 33, 64, 97, 52, 74, 67, 64,
       49, 41, 31, 75, 22, 30, 68, 75, 74, 37, 15, 30,  0, 24, 90, 87, 23,
       71, 91, 35,  2, 13, 22, 52, 65, 63, 69, 49, 70, 49, 57,  3])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'absoluto del mobile hoy en di...\n @takag talvez mis genes son recesivos y mis hi...\n @yoruguayo @rsi'

Next Char Predictions:
 b"fW2{0QW\x7flU9\nnVv[UNK]&dpBj\\/%6\\*[UNK]>^+_`!/y(f(~nPRu&dbT[UNK]9Pl9I'N;8^D^>]~Qg`]NF<h3;ahgB,;5wt4dx@\n*3Q^\\bNcNV "


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (256, 100, 99)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.59632


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(mean_loss).numpy()

99.118904

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/Shareddrives/CSCI 5523 Group Project/Models/Generative Model/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
4755/4755 [==============================] - 557s 117ms/step - loss: 1.9731
Epoch 2/30
4755/4755 [==============================] - 546s 115ms/step - loss: 1.4650
Epoch 3/30
4755/4755 [==============================] - 556s 117ms/step - loss: 1.4215
Epoch 4/30
4755/4755 [==============================] - 546s 115ms/step - loss: 1.4038
Epoch 5/30
4755/4755 [==============================] - 553s 116ms/step - loss: 1.3938
Epoch 6/30
4755/4755 [==============================] - 557s 117ms/step - loss: 1.3871
Epoch 7/30
4755/4755 [==============================] - 549s 115ms/step - loss: 1.3831
Epoch 8/30
4755/4755 [==============================] - 556s 117ms/step - loss: 1.3793
Epoch 9/30
4755/4755 [==============================] - 546s 115ms/step - loss: 1.3768
Epoch 10/30
4755/4755 [==============================] - 555s 117ms/step - loss: 1.3729
Epoch 11/30
4755/4755 [==============================] - 546s 115ms/step - loss: 1.3704
Epoch 12/30
4755/4755 [==================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    print(predicted_logits)
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
out_path = "/content/drive/Shareddrives/CSCI 5523 Group Project/Models/Generative Model/rnn_full_filtered_legitimate_tweets"
tf.saved_model.save(one_step_model, out_path)

Run the code in a loop to generate some text

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['@'])
result = [next_char]

for n in range(240):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n' + '_'*5)
print('\nRun time:', end - start)

In [ ]:
# extract string output from generated tensor
l = [x.decode('utf-8') for x in result.numpy()]
s = l[0].split('\n')

In [ ]:
s

['@paul_irasti Yessssss :P Nick :)',
 '                   I want Rachel Justin Bieber!!!!',
 "         @GJ3312 At lunch its tyler I'll go to med",
 '                              @PSBoneUSC naw it is',
 '                     @bieber_n1 argh awww =) enforce',
 ' @']

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Taylor', 'Taylor', 'Taylor', 'Taylor', 'Taylor'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

Tensor("strided_slice:0", shape=(5, 99), dtype=float32)
Tensor("strided_slice:0", shape=(5, 99), dtype=float32)
tf.Tensor(
[b'Taylor Swift!!! - Real Estate Recentmores To...\n Review Of First Analyzing Volume I was dirty h...\n Classi'
 b"Taylor Lautner.\n Ok normally my unilort because it's been nice ...\n nothing thout of activats magarime n n"
 b"Taylor's :)\n                       C'mon son. Ituff is so sma.\n                                    OMG ste"
 b"Taylors.org:))))\n @lanaessie great night! I'm freezing! They onl...\n @iMakNornikki :'( :D Its not the day "
 b'Taylor: Kellan Muscor: http...\n get an emo magic video has a cute monkey....no...\n       @aussie hello! :)'], shape=(5,), dtype=string) 

________________________________________________________________________________

Run time: 0.7638649940490723


## Restore Model and Generate Examples

In [ ]:
out_path = "/content/drive/Shareddrives/CSCI 5523 Group Project/Models/Generative Model/rnn_full_filtered_legitimate_tweets"
#tf.saved_model.save(one_step_model, out_path)
one_step_reloaded = tf.saved_model.load(out_path)

In [ ]:
states = None
first_char = tf.constant(['T', 'R', 'I', 'S', 'A']) # top 5 characters tweets start with
next_char = first_char
result = [next_char]
generated_tweets = []
counter = 0

# generate 1000 example tweets of length 140
while counter < 200:
  counter += 1
  for n in range(140):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)
  result = tf.strings.join(result)
  new_tweets = [x.decode('utf-8') for x in result.numpy()]
  generated_tweets = generated_tweets + new_tweets
  result = [first_char]

print(len(generated_tweets))

1000


In [ ]:
# save generated tweets
labels = np.ones((1000,)).astype(int)
gen_tweets_df = pd.DataFrame({'generated tweets':generated_tweets,'label':labels})
gen_tweets_df.to_csv('/content/drive/Shareddrives/CSCI 5523 Group Project/Data/GAN data/generated_tweets_1000.csv', header=True)

In [4]:
# load generated tweets
gen_tweets_df = pd.read_csv('/content/drive/Shareddrives/CSCI 5523 Group Project/Data/GAN data/generated_tweets_1000.csv', header = 0)

## Evaluate GAN Performance
Using our best polluter tweet classifier, evaluate the performance of the text generator.

In [ ]:
!pip install -q tensorflow-text

     |████████████████████████████████| 3.4MB 9.8MB/s 


In [ ]:
!pip install -q tf-models-official

     |████████████████████████████████| 1.1MB 12.9MB/s 
     |████████████████████████████████| 174kB 56.1MB/s 
     |████████████████████████████████| 358kB 53.5MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 37.6MB 136kB/s 
     |████████████████████████████████| 1.2MB 63.5MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 706kB 43.9MB/s 
     |████████████████████████████████| 645kB 50.1MB/s 


In [ ]:
import tensorflow_hub as hub
from tensorflow_hub import KerasLayer
import tensorflow_text as text
classification_model = tf.keras.models.load_model('/content/drive/Shareddrives/CSCI 5523 Group Project/Models/bert.h5', custom_objects={'KerasLayer':KerasLayer})

In [ ]:
from official.nlp import optimization  # to create AdamW optmizer

epochs = 5
steps_per_epoch = 250
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)
init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
classification_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
classification_model.evaluate(x=gen_tweets_df['generated tweets'], y=gen_tweets_df['label'])

32/32 [==============================] - 3s 66ms/step - loss: 2.7535 - binary_accuracy: 0.0053 - precision_1: 0.9091 - recall_1: 0.0053


[2.7102131843566895, 0.007000000216066837, 1.0, 0.007000000216066837]

In [6]:
# Apply tokenization and vectorization
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvec = TfidfVectorizer(ngram_range=(1,1), stop_words='english', max_features=2**13)

# Fit the data and then return the matrix
X = tfidfvec.fit_transform(gen_tweets_df['generated tweets'])

# test set size of 20% of the data and the random seed 1
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), gen_tweets_df['label'], test_size=0.2, random_state=1)

In [7]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)
predictions = naive_bayes.predict(X_test)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: {}'.format(accuracy_score(y_test, predictions)))
print('Precision score: {}'.format(precision_score(y_test, predictions)))
print('Recall score: {}'.format(recall_score(y_test, predictions)))
print('F1 score: {}'.format(f1_score(y_test, predictions)))

Accuracy score: 1.0
Precision score: 1.0
Recall score: 1.0
F1 score: 1.0
